## 랜덤포레스트(Random Forest) 모델 개발 워크플로우 (보팅/배깅 추가)

---

## 1. 모듈 및 데이터 로딩
- **설명**: 필요한 라이브러리 임포트 및 데이터셋 불러오기
- **랜덤포레스트 특화 가이드라인**:
  - 기본 라이브러리: `pandas`, `numpy`, `scikit-learn`, `matplotlib`, `seaborn`
  - 랜덤포레스트 관련: `from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor`
  - 병렬 처리: `n_jobs=-1` 사용으로 모든 CPU 코어 활용 가능
  - 피쳐 중요도 시각화: `matplotlib`, `seaborn` 필수
  - 대용량 데이터: `dask`, `polars` 고려 (랜덤포레스트는 메모리 많이 사용)

---

## 2. 데이터 전처리
- **설명**: 결측치, 이상치, 중복값 처리 및 데이터 타입 변환
- **랜덤포레스트 특화 가이드라인**:
- **결측치 처리**:
  - **결측치가 5% 이하**: 단순 대체 (평균/중앙값/최빈값) 가능
  - **결측치가 5~30%**: `SimpleImputer` 사용 (평균: mean, 중앙값: median, 최빈값: most_frequent)
  - **결측치가 30% 이상**: 해당 피쳐 제거 검토
  - **타겟 변수의 결측치**: 해당 행 제거 필수
  - ⭐ **랜덤포레스트의 장점**: 트리 기반 모델이므로 결측치 처리가 비교적 유연함
- **이상치 처리**:
  - ⭐ **랜덤포레스트는 이상치에 강건**: 제거할 필요 없음
  - 선택적 제거: 데이터 품질 문제인 경우만 제거 (실제 극단값은 보존 권장)
  - Z-score나 IQR 방법으로 탐지하되, 반드시 제거해야 할 필요 없음
- **중복값 처리**: `drop_duplicates()` 사용, 전체 행이 같은 경우만 제거
- **데이터 타입 변환**:
  - 범주형: 숫자로 변환 필수 (Label Encoding 또는 One-Hot)
  - 날짜: `datetime64` 타입으로 변환 후 연도, 월, 일 등으로 분해

---

## 3. 데이터 탐색 및 분석(EDA)
- **설명**: 데이터의 분포, 통계, 상관관계 파악 및 시각화
- **랜덤포레스트 특화 가이드라인**:
  - 기술통계: `describe()`, `info()`, `value_counts()` 확인
  - 분포 시각화: 히스토그램, 박스플롯, 바이올린플롯
  - ⭐ **상관관계**: 랜덤포레스트는 피쳐 간 상관관계에 민감하지 않으므로 참고용
  - 타겟 변수 불균형 확인: 분류 문제일 경우 클래스 비율 반드시 확인
    - 불균형 데이터 대응: `class_weight='balanced'` 또는 `class_weight='balanced_subsample'` 사용
  - 피쳐 분포: 편포된 데이터도 괜찮음 (스케일링 불필요)

---

## 4. 인코딩
- **설명**: 범주형 변수를 수치형으로 변환
- **랜덤포레스트 특화 인코더 선택 가이드라인**:

| 상황 | 추천 인코더 | 이유 |
|---|---|---|
| **범주형 변수** (일반적) | **Label Encoding** | 랜덤포레스트는 순서 무관, 효율적 |
| **카테고리 수 < 10개** | **Label Encoding** | 단순, 빠름, 메모리 절약 |
| **카테고리 수 > 100개** (고카디널리티) | **Label Encoding** | One-Hot은 메모리 낭비 |
| **순서 있는 범주형** (낮음, 중간, 높음) | **Ordinal Encoding** | 순서 정보 활용 가능 |
| **타겟 변수** | **Label Encoding** | 각 클래스에 정수 할당 (0, 1, 2...) |
| **피처 중요도 해석 필요** | **원본 범주명 보존** | 인코딩 매핑표 저장해서 나중에 참고 |

- **인코딩 전략**:
  - 트리 기반 모델: One-Hot Encoding 불필요 (오히려 메모리 낭비)
  - Label Encoding으로 충분: 0, 1, 2, ... 번호 할당

```python
from sklearn.preprocessing import LabelEncoder

# 예시
le_color = LabelEncoder()
X['color'] = le_color.fit_transform(X['color'])
# 매핑 저장: {'red': 0, 'blue': 1, 'green': 2}
```

---

## 5. 피쳐 엔지니어링
- **설명**: 새로운 피쳐 생성, 피쳐 선택 및 변환
- **랜덤포레스트 특화 가이드라인**:
  - **새로운 피쳐 생성**: 도메인 지식 활용 (예: 나이에서 연령대 범주 생성)
  - **다항 피쳐**: 필요 없음 (랜덤포레스트가 상호작용 학습 가능)
  - **스케일링**: 불필요 (트리 기반이므로 절대값 무관)
  - ⭐ **피쳐 선택 방법**:
    - **트리 기반 피쳐 중요도**: 모델 학습 후 `feature_importances_` 속성 활용
    - **Permutation Importance**: 더 신뢰할 수 있는 피쳐 중요도
    - **SHAP 값**: 각 피쳐의 개별 영향도 분석
  - **피쳐 제거**: 중요도 낮은 피쳐 제거해서 모델 단순화 가능

---

## 6. 피쳐 및 타겟 분리
- **설명**: X (독립변수)와 y (종속변수) 분리
- **랜덤포레스트 특화 가이드라인**:
  - 타겟 변수는 반드시 y에 할당
  - 피쳐는 X에 할당 (타겟 제외, ID 열 제외, 날짜 원본 제외)
  - 범주형 변수는 반드시 수치형으로 인코딩 후 포함

---

## 7. 데이터셋 분할
- **설명**: 학습 데이터와 테스트 데이터 분리
- **랜덤포레스트 특화 가이드라인**:
  - **기본 분할**: 70/30 또는 80/20 비율
  - **불균형 분류 데이터**: `stratify=y` 옵션으로 클래스 비율 유지 (필수)
  - **시계열 데이터**: 시간 순서대로 분할 (무작위 분할 금지)
  - **소규모 데이터 (< 1000개)**: 더 큰 비율로 학습 데이터 할당 (90/10)
  - ⭐ **랜덤포레스트의 특징**: 많은 데이터 필요 (나무 개수 * 스플릿마다 샘플링)

```python
from sklearn.model_selection import train_test_split

# 분류 문제 (불균형 데이터)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 회귀 문제
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)
```

---

## 8. 피쳐 스케일링
- **설명**: 피쳐를 동일한 스케일로 정규화 또는 표준화
- **랜덤포레스트 특화 가이드라인**:

⭐ **랜덤포레스트는 스케일링 불필요**

| 이유 | 설명 |
|---|---|
| **트리 기반 모델** | 절대값 비교하지 않음, 대소 비교만 수행 |
| **분할 기준** | 각 노드에서 "값 > 임계값?" 형태의 질문만 함 |
| **메모리 절약** | 스케일링 생략으로 계산 시간 단축 |
| **특성 보존** | 원본 스케일 유지하면 해석 용이 |

- **예외 상황**:
  - 다른 모델과 비교할 때는 공정성 위해 스케일링 가능
  - 시각화 목적으로 스케일링 가능

---

## 9. 머신러닝 모델 선정 및 학습
- **설명**: 랜덤포레스트 모델 선택 및 초기 학습
- **랜덤포레스트 특화 가이드라인**:

### 분류(Classification) 문제

```python
from sklearn.ensemble import RandomForestClassifier

# 기본 설정
model = RandomForestClassifier(
    n_estimators=100, # 나무 개수 (기본값)
    max_depth=None, # 최대 깊이 (기본: 제한 없음, 튜닝 권장)
    min_samples_split=2, # 분할 최소 샘플 수 (기본값)
    min_samples_leaf=1, # 리프 노드 최소 샘플 수 (기본값)
    random_state=42, # 재현성 위해 고정
    n_jobs=-1, # 모든 CPU 코어 사용
    class_weight='balanced' # 불균형 클래스 처리
)

model.fit(X_train, y_train)
```

### 회귀(Regression) 문제

```python
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    n_estimators=100,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)
```

| 상황 | 추천 설정 | 이유 |
|---|---|---|
| **작은 데이터** (< 10K) | n_estimators=50, max_depth=10 | 과적합 방지 |
| **중간 크기** (10K~100K) | n_estimators=100, max_depth=15 | 균형잡힌 성능 |
| **대용량 데이터** (> 100K) | n_estimators=200, max_depth=20 | 충분한 학습 |
| **높은 정확도 필요** | n_estimators=300~500 | 더 많은 나무 = 더 안정적 |
| **빠른 학습 필요** | n_estimators=50, max_depth=10 | 계산 시간 단축 |

---

## 10. 앙상블 확장: 보팅(Voting) 및 배깅(Bagging) 선택
- **설명**: 단일 랜덤포레스트 모델을 넘어, 여러 모델을 결합하는 보팅 또는 다른 배깅 방식을 고려하여 성능을 극대화하는 단계입니다.
- **가이드라인**:

### 앙상블 기법 선택 가이드

| 기법 | 설명 | 언제 사용하는가? |
|---|---|---|
| **랜덤포레스트 (단독)** | **배깅(Bagging)의 한 종류**. 결정 트리를 기반으로 여러 모델을 만들어 결과를 종합합니다. | 빠르고 강력한 성능이 필요할 때. 대부분의 경우 좋은 시작점입니다. |
| **배깅 (Bagging)** | 랜덤포레스트 외 **다른 모델(예: SVM, 로지스틱 회귀)을 배깅**하고 싶을 때 사용합니다. | 결정 트리가 아닌 다른 기본 모델의 분산을 줄이고 싶을 때 적합합니다. |
| **보팅 (Voting)** | **서로 다른 유형의 여러 모델**(예: 랜덤포레스트, Gradient Boosting, SVM)의 예측을 결합합니다. | 최고 수준의 성능을 원할 때. 각 모델의 단점을 서로 보완하여 더 안정적이고 높은 정확도를 기대할 수 있습니다. |

### 보팅 (Voting) 구현 예시
- 서로 다른 모델(예: 랜덤포레스트, 로지스틱 회귀, Gradient Boosting)을 학습시킨 후, 예측 결과를 투표를 통해 종합합니다.
- **Soft Voting**: 각 모델의 예측 확률의 평균을 내어 최종 예측. 일반적으로 성능이 더 좋음.
- **Hard Voting**: 다수결 원칙으로 가장 많이 예측된 클래스를 선택.

```python
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

# 1. 개별 모델 정의
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
lr_clf = LogisticRegression(random_state=42)
gb_clf = GradientBoostingClassifier(random_state=42)

# 2. 보팅 분류기 생성
# soft voting을 위해선 모든 모델이 predict_proba를 지원해야 함
voting_clf = VotingClassifier(
    estimators=[('rf', rf_clf), ('lr', lr_clf), ('gb', gb_clf)],
    voting='soft', # 확률 기반 투표
    n_jobs=-1
)

# 3. 보팅 모델 학습
voting_clf.fit(X_train, y_train)

# 4. 평가
voting_pred = voting_clf.predict(X_test)
print(f"보팅 분류기 정확도: {accuracy_score(y_test, voting_pred):.4f}")
```

---

## 11. 교차 검증(Cross-Validation)
- **설명**: K-Fold로 모델 성능 검증 및 과적합 방지
- **랜덤포레스트 특화 가이드라인**:
  - **기본 K-Fold**: K=5 또는 K=10 (데이터 크기에 따라)
  - **소규모 데이터 (< 100개)**: K=3 또는 Leave-One-Out CV (LOOCV)
  - ⭐ **불균형 데이터 (필수)**: **StratifiedKFold** 사용 (클래스 비율 유지)
  - **시계열 데이터**: **TimeSeriesSplit** 사용 (시간 순서 유지)
  - **평가 지표**: 평균 점수와 표준편차 모두 확인 (분산 측정)

```python
from sklearn.model_selection import cross_val_score, StratifiedKFold

# 분류 문제 (랜덤포레스트 단일 모델)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X_train, y_train, cv=skf, scoring='accuracy')
print(f"랜덤포레스트 평균 점수: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

# 보팅 모델 교차 검증
cv_scores_voting = cross_val_score(voting_clf, X_train, y_train, cv=skf, scoring='accuracy')
print(f"보팅 분류기 평균 점수: {cv_scores_voting.mean():.4f} (+/- {cv_scores_voting.std():.4f})")
```

---

## 12. 하이퍼파라미터 튜닝
- **설명**: GridSearchCV 또는 RandomizedSearchCV로 최적 파라미터 선택
- **랜덤포레스트 특화 가이드라인**:

### 튜닝 전략

| 방법 | 파라미터 개수 | 추천 상황 | 비용 |
|---|---|---|---|
| **Grid Search** | ≤ 3개 | 간단한 튜닝 | 높음 |
| **Random Search** | > 3개 | 대부분의 경우 (권장) | 낮음 |
| **Bayesian Optimization**| 무제한 | 최고 성능 필요 | 중간 |

### 주요 하이퍼파라미터 튜닝 (우선순위 높음)

**1순위: n_estimators (나무 개수)**
`'n_estimators': [50, 100, 200, 300, 500]`
**2순위: max_depth (최대 깊이)**
`'max_depth': [5, 10, 15, 20, None]`
**3순위: min_samples_split (분할 최소 샘플)**
`'min_samples_split': [2, 5, 10, 20]`
**4순위: min_samples_leaf (리프 최소 샘플)**
`'min_samples_leaf': [1, 2, 4, 8]`
**5순위: max_features (분할 시 고려할 피쳐 수)**
`'max_features': ['sqrt', 'log2', None]`

### RandomizedSearchCV 사용 예시 (권장)

```python
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'n_estimators': [50, 100, 200, 300, 500],
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': ['sqrt', 'log2']
}

random_search = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42, n_jobs=-1),
    param_distributions=param_dist,
    n_iter=20, # 20개 조합 샘플링
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

random_search.fit(X_train, y_train)
print(f"최적 파라미터: {random_search.best_params_}")
print(f"최고 점수: {random_search.best_score_:.4f}")
```

---

## 13. 최종 모델 생성
- **설명**: 최적 파라미터로 전체 학습 데이터에서 재학습
- **랜덤포레스트 특화 가이드라인**:

```python
# 그리드 서치 결과로부터 최적 모델 추출
best_model = random_search.best_estimator_

# 또는 수동으로 최적 파라미터로 새로 생성
best_model = RandomForestClassifier(
    n_estimators=random_search.best_params_['n_estimators'],
    max_depth=random_search.best_params_['max_depth'],
    # ... 다른 최적 파라미터들
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)

# 전체 학습 데이터로 재학습
best_model.fit(X_train, y_train)

# 모델 저장
import joblib
joblib.dump(best_model, 'best_random_forest_model.pkl')
```

- **가이드라인**:
  - 튜닝된 하이퍼파라미터로 모델 재생성
  - 전체 학습 데이터 (X_train, y_train) 사용하여 학습
  - 모델 저장: `joblib.dump()` 또는 `pickle`

---

## 14. 테스트 데이터 성능 평가
- **설명**: 평가 메트릭으로 모델 성능 측정
- **랜덤포레스트 특화 가이드라인**:

### 분류 문제 평가 지표

```python
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix,
    classification_report
)

y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)

# 기본 메트릭
print(f"정확도 (Accuracy): {accuracy_score(y_test, y_pred):.4f}")
print(f"정밀도 (Precision): {precision_score(y_test, y_pred, average='weighted'):.4f}")
print(f"재현율 (Recall): {recall_score(y_test, y_pred, average='weighted'):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred, average='weighted'):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba, multi_class='ovr'):.4f}")
```

### 회귀 문제 평가 지표

```python
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_pred = best_model.predict(X_test)

print(f"MSE: {mean_squared_error(y_test, y_pred):.4f}")
print(f"RMSE: {mean_squared_error(y_test, y_pred, squared=False):.4f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.4f}")
print(f"R²: {r2_score(y_test, y_pred):.4f}")
```

---

## 15. 피쳐 중요도 분석 (랜덤포레스트 특화)

⭐ **랜덤포레스트의 강점: 피쳐 중요도 자동 제공**

```python
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 피쳐 중요도 추출
feature_importance = best_model.feature_importances_
feature_names = X_train.columns

# 데이터프레임으로 정렬
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

# 시각화 (상위 10개)
plt.figure(figsize=(12, 8))
sns.barplot(x='importance', y='feature', data=importance_df.head(10))
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Random Forest - Top 10 Important Features')
plt.show()
```

---

## 16. 데이터 예측
- **설명**: 새로운 데이터에 대한 예측 수행
- **랜덤포레스트 특화 가이드라인**:

```python
# 새로운 데이터 준비 (학습 데이터와 동일한 전처리)
new_data = pd.DataFrame({...})

# 범주형 변수 인코딩 (학습 시 사용한 인코더 재사용)
new_data['color'] = le_color.transform(new_data['color'])

# 예측 (분류)
predictions = best_model.predict(new_data)
probabilities = best_model.predict_proba(new_data)

# 예측 결과 확인
result_df = pd.DataFrame({
    'prediction': predictions,
    'confidence': probabilities.max(axis=1)
})

print(result_df)
```

---

## 최종 워크플로우 요약 (랜덤포레스트)

```
1. 모듈 및 데이터 로딩
   ↓ (n_jobs=-1로 병렬 처리 설정)
2. 데이터 전처리
   ↓ (결측치/이상치 처리, Label Encoding)
3. EDA
   ↓ (불균형 데이터 확인)
4. 인코딩 (범주형 변수를 Label Encoding으로)
   ↓
5. 피쳐 엔지니어링
   ↓ (스케일링 불필요)
6. 피쳐/타겟 분리
   ↓
7. 데이터셋 분할
   ↓ (불균형이면 stratify=y 필수)
8. 피쳐 스케일링 (랜덤포레스트는 불필요 - 생략)
   ↓
9. 모델 선정 및 학습
   ↓ (n_estimators=100, max_depth=None 기본값)
10. 앙상블 확장 고려 (보팅/배깅)
   ↓ (성능 극대화를 위해 다른 모델과 결합)
11. 교차 검증
   ↓ (StratifiedKFold 사용)
12. 하이퍼파라미터 튜닝
   ↓ (RandomizedSearchCV 권장)
13. 최종 모델 생성
   ↓ (최적 파라미터로 재학습)
14. 테스트 성능 평가
   ↓ (Accuracy, Precision, Recall, F1-Score)
15. 피쳐 중요도 분석
   ↓ (feature_importances_ 확인)
16. 데이터 예측
   ↓ (predict_proba로 확률도 함께 제공)
```

---

## 랜덤포레스트 핵심 특징 정리

| 특징 | 설명 | 활용 |
|---|---|---|
| **강건성** | 이상치, 스케일링에 민감하지 않음 | 전처리 간소화 |
| **피쳐 중요도**| 자동으로 각 피쳐 중요도 계산 | 해석 가능성 높음 |
| **병렬 처리** | n_jobs=-1로 모든 CPU 활용 | 빠른 학습 가능 |
| **앙상블** | 여러 트리의 투표로 정확도 향상 | 과적합 방지 |
| **유연성** | 분류, 회귀 모두 가능 | 다양한 문제 적용 |
| **과적합 저항**| max_depth 튜닝으로 제어 가능 | 일반화 능력 우수 |

---

## 주의사항

1. **메모리 사용**: n_estimators가 많을수록 메모리 많이 사용
2. **계산 시간**: 데이터가 많으면 학습에 오래 걸림 (병렬 처리로 단축)
3. **과적합**: max_depth를 제한해야 과적합 방지
4. **불균형 데이터**: class_weight='balanced' 또는 stratify=y 필수
5. **재현성**: random_state 고정으로 결과 재현 가능하게

데이터의 특성과 문제 유형에 따라 일부 단계는 반복하거나 순서를 조정할 수 있습니다.


# 랜덤포레스트(Random Forest) 모델 개발 워크플로우 (보팅/배깅 추가)

---

## 1. 모듈 및 데이터 로딩
- **설명**: 필요한 라이브러리 임포트 및 데이터셋 불러오기
- **랜덤포레스트 특화 가이드라인**:
  - 기본 라이브러리: `pandas`, `numpy`, `scikit-learn`, `matplotlib`, `seaborn`
  - 랜덤포레스트 관련: `from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor`
  - 병렬 처리: `n_jobs=-1` 사용으로 모든 CPU 코어 활용 가능
  - 피쳐 중요도 시각화: `matplotlib`, `seaborn` 필수
  - 대용량 데이터: `dask`, `polars` 고려 (랜덤포레스트는 메모리 많이 사용)

---

## 2. 데이터 전처리
- **설명**: 결측치, 이상치, 중복값 처리 및 데이터 타입 변환
- **랜덤포레스트 특화 가이드라인**:
- **결측치 처리**:
  - **결측치가 5% 이하**: 단순 대체 (평균/중앙값/최빈값) 가능
  - **결측치가 5~30%**: `SimpleImputer` 사용 (평균: mean, 중앙값: median, 최빈값: most_frequent)
  - **결측치가 30% 이상**: 해당 피쳐 제거 검토
  - **타겟 변수의 결측치**: 해당 행 제거 필수
  - ⭐ **랜덤포레스트의 장점**: 트리 기반 모델이므로 결측치 처리가 비교적 유연함
- **이상치 처리**:
  - ⭐ **랜덤포레스트는 이상치에 강건**: 제거할 필요 없음
  - 선택적 제거: 데이터 품질 문제인 경우만 제거 (실제 극단값은 보존 권장)
  - Z-score나 IQR 방법으로 탐지하되, 반드시 제거해야 할 필요 없음
- **중복값 처리**: `drop_duplicates()` 사용, 전체 행이 같은 경우만 제거
- **데이터 타입 변환**:
  - 범주형: 숫자로 변환 필수 (Label Encoding 또는 One-Hot)
  - 날짜: `datetime64` 타입으로 변환 후 연도, 월, 일 등으로 분해

---

## 3. 데이터 탐색 및 분석(EDA)
- **설명**: 데이터의 분포, 통계, 상관관계 파악 및 시각화
- **랜덤포레스트 특화 가이드라인**:
  - 기술통계: `describe()`, `info()`, `value_counts()` 확인
  - 분포 시각화: 히스토그램, 박스플롯, 바이올린플롯
  - ⭐ **상관관계**: 랜덤포레스트는 피쳐 간 상관관계에 민감하지 않으므로 참고용
  - 타겟 변수 불균형 확인: 분류 문제일 경우 클래스 비율 반드시 확인
    - 불균형 데이터 대응: `class_weight='balanced'` 또는 `class_weight='balanced_subsample'` 사용
  - 피쳐 분포: 편포된 데이터도 괜찮음 (스케일링 불필요)

---

## 4. 인코딩
- **설명**: 범주형 변수를 수치형으로 변환
- **랜덤포레스트 특화 인코더 선택 가이드라인**:

| 상황 | 추천 인코더 | 이유 |
|---|---|---|
| **범주형 변수** (일반적) | **Label Encoding** | 랜덤포레스트는 순서 무관, 효율적 |
| **카테고리 수 < 10개** | **Label Encoding** | 단순, 빠름, 메모리 절약 |
| **카테고리 수 > 100개** (고카디널리티) | **Label Encoding** | One-Hot은 메모리 낭비 |
| **순서 있는 범주형** (낮음, 중간, 높음) | **Ordinal Encoding** | 순서 정보 활용 가능 |
| **타겟 변수** | **Label Encoding** | 각 클래스에 정수 할당 (0, 1, 2...) |
| **피처 중요도 해석 필요** | **원본 범주명 보존** | 인코딩 매핑표 저장해서 나중에 참고 |

- **인코딩 전략**:
  - 트리 기반 모델: One-Hot Encoding 불필요 (오히려 메모리 낭비)
  - Label Encoding으로 충분: 0, 1, 2, ... 번호 할당

```python
from sklearn.preprocessing import LabelEncoder

# 예시
le_color = LabelEncoder()
X['color'] = le_color.fit_transform(X['color'])
# 매핑 저장: {'red': 0, 'blue': 1, 'green': 2}
```

---

## 5. 피쳐 엔지니어링
- **설명**: 새로운 피쳐 생성, 피쳐 선택 및 변환
- **랜덤포레스트 특화 가이드라인**:
  - **새로운 피쳐 생성**: 도메인 지식 활용 (예: 나이에서 연령대 범주 생성)
  - **다항 피쳐**: 필요 없음 (랜덤포레스트가 상호작용 학습 가능)
  - **스케일링**: 불필요 (트리 기반이므로 절대값 무관)
  - ⭐ **피쳐 선택 방법**:
    - **트리 기반 피쳐 중요도**: 모델 학습 후 `feature_importances_` 속성 활용
    - **Permutation Importance**: 더 신뢰할 수 있는 피쳐 중요도
    - **SHAP 값**: 각 피쳐의 개별 영향도 분석
  - **피쳐 제거**: 중요도 낮은 피쳐 제거해서 모델 단순화 가능

---

## 6. 피쳐 및 타겟 분리
- **설명**: X (독립변수)와 y (종속변수) 분리
- **랜덤포레스트 특화 가이드라인**:
  - 타겟 변수는 반드시 y에 할당
  - 피쳐는 X에 할당 (타겟 제외, ID 열 제외, 날짜 원본 제외)
  - 범주형 변수는 반드시 수치형으로 인코딩 후 포함

---

## 7. 데이터셋 분할
- **설명**: 학습 데이터와 테스트 데이터 분리
- **랜덤포레스트 특화 가이드라인**:
  - **기본 분할**: 70/30 또는 80/20 비율
  - **불균형 분류 데이터**: `stratify=y` 옵션으로 클래스 비율 유지 (필수)
  - **시계열 데이터**: 시간 순서대로 분할 (무작위 분할 금지)
  - **소규모 데이터 (< 1000개)**: 더 큰 비율로 학습 데이터 할당 (90/10)
  - ⭐ **랜덤포레스트의 특징**: 많은 데이터 필요 (나무 개수 * 스플릿마다 샘플링)

```python
from sklearn.model_selection import train_test_split

# 분류 문제 (불균형 데이터)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 회귀 문제
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)
```

---

## 8. 피쳐 스케일링
- **설명**: 피쳐를 동일한 스케일로 정규화 또는 표준화
- **랜덤포레스트 특화 가이드라인**:

⭐ **랜덤포레스트는 스케일링 불필요**

| 이유 | 설명 |
|---|---|
| **트리 기반 모델** | 절대값 비교하지 않음, 대소 비교만 수행 |
| **분할 기준** | 각 노드에서 "값 > 임계값?" 형태의 질문만 함 |
| **메모리 절약** | 스케일링 생략으로 계산 시간 단축 |
| **특성 보존** | 원본 스케일 유지하면 해석 용이 |

- **예외 상황**:
  - 다른 모델과 비교할 때는 공정성 위해 스케일링 가능
  - 시각화 목적으로 스케일링 가능

---

## 9. 머신러닝 모델 선정 및 학습
- **설명**: 랜덤포레스트 모델 선택 및 초기 학습
- **랜덤포레스트 특화 가이드라인**:

### 분류(Classification) 문제

```python
from sklearn.ensemble import RandomForestClassifier

# 기본 설정
model = RandomForestClassifier(
    n_estimators=100, # 나무 개수 (기본값)
    max_depth=None, # 최대 깊이 (기본: 제한 없음, 튜닝 권장)
    min_samples_split=2, # 분할 최소 샘플 수 (기본값)
    min_samples_leaf=1, # 리프 노드 최소 샘플 수 (기본값)
    random_state=42, # 재현성 위해 고정
    n_jobs=-1, # 모든 CPU 코어 사용
    class_weight='balanced' # 불균형 클래스 처리
)

model.fit(X_train, y_train)
```

### 회귀(Regression) 문제

```python
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    n_estimators=100,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)
```

| 상황 | 추천 설정 | 이유 |
|---|---|---|
| **작은 데이터** (< 10K) | n_estimators=50, max_depth=10 | 과적합 방지 |
| **중간 크기** (10K~100K) | n_estimators=100, max_depth=15 | 균형잡힌 성능 |
| **대용량 데이터** (> 100K) | n_estimators=200, max_depth=20 | 충분한 학습 |
| **높은 정확도 필요** | n_estimators=300~500 | 더 많은 나무 = 더 안정적 |
| **빠른 학습 필요** | n_estimators=50, max_depth=10 | 계산 시간 단축 |

---

## 10. 앙상블 확장: 보팅(Voting) 및 배깅(Bagging) 선택
- **설명**: 단일 랜덤포레스트 모델을 넘어, 여러 모델을 결합하는 보팅 또는 다른 배깅 방식을 고려하여 성능을 극대화하는 단계입니다.
- **가이드라인**:

### 앙상블 기법 선택 가이드

| 기법 | 설명 | 언제 사용하는가? |
|---|---|---|
| **랜덤포레스트 (단독)** | **배깅(Bagging)의 한 종류**. 결정 트리를 기반으로 여러 모델을 만들어 결과를 종합합니다. | 빠르고 강력한 성능이 필요할 때. 대부분의 경우 좋은 시작점입니다. |
| **배깅 (Bagging)** | 랜덤포레스트 외 **다른 모델(예: SVM, 로지스틱 회귀)을 배깅**하고 싶을 때 사용합니다. | 결정 트리가 아닌 다른 기본 모델의 분산을 줄이고 싶을 때 적합합니다. |
| **보팅 (Voting)** | **서로 다른 유형의 여러 모델**(예: 랜덤포레스트, Gradient Boosting, SVM)의 예측을 결합합니다. | 최고 수준의 성능을 원할 때. 각 모델의 단점을 서로 보완하여 더 안정적이고 높은 정확도를 기대할 수 있습니다. |

### 보팅 (Voting) 구현 예시
- 서로 다른 모델(예: 랜덤포레스트, 로지스틱 회귀, Gradient Boosting)을 학습시킨 후, 예측 결과를 투표를 통해 종합합니다.
- **Soft Voting**: 각 모델의 예측 확률의 평균을 내어 최종 예측. 일반적으로 성능이 더 좋음.
- **Hard Voting**: 다수결 원칙으로 가장 많이 예측된 클래스를 선택.

```python
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

# 1. 개별 모델 정의
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
lr_clf = LogisticRegression(random_state=42)
gb_clf = GradientBoostingClassifier(random_state=42)

# 2. 보팅 분류기 생성
# soft voting을 위해선 모든 모델이 predict_proba를 지원해야 함
voting_clf = VotingClassifier(
    estimators=[('rf', rf_clf), ('lr', lr_clf), ('gb', gb_clf)],
    voting='soft', # 확률 기반 투표
    n_jobs=-1
)

# 3. 보팅 모델 학습
voting_clf.fit(X_train, y_train)

# 4. 평가
voting_pred = voting_clf.predict(X_test)
print(f"보팅 분류기 정확도: {accuracy_score(y_test, voting_pred):.4f}")
```

### 보팅 (Voting) 구현 예시 - 회귀 문제

```python
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

# 1. 개별 회귀 모델 정의
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
lr_reg = LinearRegression()
gb_reg = GradientBoostingRegressor(random_state=42)

# 2. 보팅 회귀 모델 생성
voting_reg = VotingRegressor(
    estimators=[('rf', rf_reg), ('lr', lr_reg), ('gb', gb_reg)],
    n_jobs=-1
)

# 3. 보팅 모델 학습
voting_reg.fit(X_train, y_train)

# 4. 평가
voting_pred = voting_reg.predict(X_test)
print(f"보팅 회귀 모델 R²: {r2_score(y_test, voting_pred):.4f}")
```

---

## 11. 교차 검증(Cross-Validation)
- **설명**: K-Fold로 모델 성능 검증 및 과적합 방지
- **랜덤포레스트 특화 가이드라인**:
  - **기본 K-Fold**: K=5 또는 K=10 (데이터 크기에 따라)
  - **소규모 데이터 (< 100개)**: K=3 또는 Leave-One-Out CV (LOOCV)
  - ⭐ **불균형 데이터 (필수)**: **StratifiedKFold** 사용 (클래스 비율 유지)
  - **시계열 데이터**: **TimeSeriesSplit** 사용 (시간 순서 유지)
  - **평가 지표**: 평균 점수와 표준편차 모두 확인 (분산 측정)

```python
from sklearn.model_selection import cross_val_score, StratifiedKFold

# 분류 문제 (랜덤포레스트 단일 모델)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X_train, y_train, cv=skf, scoring='accuracy')
print(f"랜덤포레스트 평균 점수: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

# 보팅 모델 교차 검증
cv_scores_voting = cross_val_score(voting_clf, X_train, y_train, cv=skf, scoring='accuracy')
print(f"보팅 분류기 평균 점수: {cv_scores_voting.mean():.4f} (+/- {cv_scores_voting.std():.4f})")
```

---

## 12. 하이퍼파라미터 튜닝
- **설명**: GridSearchCV 또는 RandomizedSearchCV로 최적 파라미터 선택
- **랜덤포레스트 특화 가이드라인**:

### 튜닝 전략

| 방법 | 파라미터 개수 | 추천 상황 | 비용 |
|---|---|---|---|
| **Grid Search** | ≤ 3개 | 간단한 튜닝 | 높음 |
| **Random Search** | > 3개 | 대부분의 경우 (권장) | 낮음 |
| **Bayesian Optimization**| 무제한 | 최고 성능 필요 | 중간 |

### 주요 하이퍼파라미터 튜닝 (우선순위 높음)

**1순위: n_estimators (나무 개수)**
`'n_estimators': [50, 100, 200, 300, 500]`
**2순위: max_depth (최대 깊이)**
`'max_depth': [5, 10, 15, 20, None]`
**3순위: min_samples_split (분할 최소 샘플)**
`'min_samples_split': [2, 5, 10, 20]`
**4순위: min_samples_leaf (리프 최소 샘플)**
`'min_samples_leaf': [1, 2, 4, 8]`
**5순위: max_features (분할 시 고려할 피쳐 수)**
`'max_features': ['sqrt', 'log2', None]`

### RandomizedSearchCV 사용 예시 (권장)

```python
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'n_estimators': [50, 100, 200, 300, 500],
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': ['sqrt', 'log2']
}

random_search = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42, n_jobs=-1),
    param_distributions=param_dist,
    n_iter=20, # 20개 조합 샘플링
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

random_search.fit(X_train, y_train)
print(f"최적 파라미터: {random_search.best_params_}")
print(f"최고 점수: {random_search.best_score_:.4f}")
```

---

## 13. 최종 모델 생성
- **설명**: 최적 파라미터로 전체 학습 데이터에서 재학습
- **랜덤포레스트 특화 가이드라인**:

```python
# 그리드 서치 결과로부터 최적 모델 추출
best_model = random_search.best_estimator_

# 또는 수동으로 최적 파라미터로 새로 생성
best_model = RandomForestClassifier(
    n_estimators=random_search.best_params_['n_estimators'],
    max_depth=random_search.best_params_['max_depth'],
    # ... 다른 최적 파라미터들
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)

# 전체 학습 데이터로 재학습
best_model.fit(X_train, y_train)

# 모델 저장
import joblib
joblib.dump(best_model, 'best_random_forest_model.pkl')
```

- **가이드라인**:
  - 튜닝된 하이퍼파라미터로 모델 재생성
  - 전체 학습 데이터 (X_train, y_train) 사용하여 학습
  - 모델 저장: `joblib.dump()` 또는 `pickle`

---

## 14. 테스트 데이터 성능 평가
- **설명**: 평가 메트릭으로 모델 성능 측정
- **랜덤포레스트 특화 가이드라인**:

### 분류 문제 평가 지표

```python
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix,
    classification_report
)

y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)

# 기본 메트릭
print(f"정확도 (Accuracy): {accuracy_score(y_test, y_pred):.4f}")
print(f"정밀도 (Precision): {precision_score(y_test, y_pred, average='weighted'):.4f}")
print(f"재현율 (Recall): {recall_score(y_test, y_pred, average='weighted'):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred, average='weighted'):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba, multi_class='ovr'):.4f}")
```

### 회귀 문제 평가 지표

```python
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_pred = best_model.predict(X_test)

print(f"MSE: {mean_squared_error(y_test, y_pred):.4f}")
print(f"RMSE: {mean_squared_error(y_test, y_pred, squared=False):.4f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.4f}")
print(f"R²: {r2_score(y_test, y_pred):.4f}")
```

---

## 15. 피쳐 중요도 분석 (랜덤포레스트 특화)

⭐ **랜덤포레스트의 강점: 피쳐 중요도 자동 제공**

```python
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 피쳐 중요도 추출
feature_importance = best_model.feature_importances_
feature_names = X_train.columns

# 데이터프레임으로 정렬
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

# 시각화 (상위 10개)
plt.figure(figsize=(12, 8))
sns.barplot(x='importance', y='feature', data=importance_df.head(10))
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Random Forest - Top 10 Important Features')
plt.show()
```

---

## 16. 데이터 예측
- **설명**: 새로운 데이터에 대한 예측 수행
- **랜덤포레스트 특화 가이드라인**:

### 분류 문제 예측

```python
# 새로운 데이터 준비 (학습 데이터와 동일한 전처리)
new_data = pd.DataFrame({...})

# 범주형 변수 인코딩 (학습 시 사용한 인코더 재사용)
new_data['color'] = le_color.transform(new_data['color'])

# 예측 (분류)
predictions = best_model.predict(new_data)
probabilities = best_model.predict_proba(new_data)

# 예측 결과 확인
result_df = pd.DataFrame({
    'prediction': predictions,
    'confidence': probabilities.max(axis=1)
})

print(result_df)
```

### 회귀 문제 예측

```python
# 새로운 데이터 준비 (학습 데이터와 동일한 전처리)
new_data = pd.DataFrame({...})

# 범주형 변수 인코딩 (학습 시 사용한 인코더 재사용)
new_data['color'] = le_color.transform(new_data['color'])

# 예측 (회귀)
predictions = best_model.predict(new_data)

# 예측 결과 데이터프레임으로 정리
result_df = pd.DataFrame({
    'predicted_value': predictions
})

print(result_df)

# 신뢰도 평가를 위해 학습 데이터에서의 예측 오차 확인
train_predictions = best_model.predict(X_train)
residuals = y_train - train_predictions

print(f"평균 절대 오차 (MAE): {abs(residuals).mean():.4f}")
print(f"표준 편차: {residuals.std():.4f}")
print(f"예측 구간: ±{residuals.std():.4f}")
```

---

## 최종 워크플로우 요약 (랜덤포레스트)

```
1. 모듈 및 데이터 로딩
   ↓ (n_jobs=-1로 병렬 처리 설정)
2. 데이터 전처리
   ↓ (결측치/이상치 처리, Label Encoding)
3. EDA
   ↓ (불균형 데이터 확인)
4. 인코딩 (범주형 변수를 Label Encoding으로)
   ↓
5. 피쳐 엔지니어링
   ↓ (스케일링 불필요)
6. 피쳐/타겟 분리
   ↓
7. 데이터셋 분할
   ↓ (불균형이면 stratify=y 필수)
8. 피쳐 스케일링 (랜덤포레스트는 불필요 - 생략)
   ↓
9. 모델 선정 및 학습
   ↓ (n_estimators=100, max_depth=None 기본값)
10. 앙상블 확장 고려 (보팅/배깅)
   ↓ (성능 극대화를 위해 다른 모델과 결합)
11. 교차 검증
   ↓ (StratifiedKFold 사용)
12. 하이퍼파라미터 튜닝
   ↓ (RandomizedSearchCV 권장)
13. 최종 모델 생성
   ↓ (최적 파라미터로 재학습)
14. 테스트 성능 평가
   ↓ (Accuracy, Precision, Recall, F1-Score)
15. 피쳐 중요도 분석
   ↓ (feature_importances_ 확인)
16. 데이터 예측
   ↓ (분류: predict_proba로 확률, 회귀: predict로 예측값)
```

---

## 랜덤포레스트 핵심 특징 정리

| 특징 | 설명 | 활용 |
|---|---|---|
| **강건성** | 이상치, 스케일링에 민감하지 않음 | 전처리 간소화 |
| **피쳐 중요도**| 자동으로 각 피쳐 중요도 계산 | 해석 가능성 높음 |
| **병렬 처리** | n_jobs=-1로 모든 CPU 활용 | 빠른 학습 가능 |
| **앙상블** | 여러 트리의 투표로 정확도 향상 | 과적합 방지 |
| **유연성** | 분류, 회귀 모두 가능 | 다양한 문제 적용 |
| **과적합 저항**| max_depth 튜닝으로 제어 가능 | 일반화 능력 우수 |

---

## 주의사항

1. **메모리 사용**: n_estimators가 많을수록 메모리 많이 사용
2. **계산 시간**: 데이터가 많으면 학습에 오래 걸림 (병렬 처리로 단축)
3. **과적합**: max_depth를 제한해야 과적합 방지
4. **불균형 데이터**: class_weight='balanced' 또는 stratify=y 필수
5. **재현성**: random_state 고정으로 결과 재현 가능하게

데이터의 특성과 문제 유형에 따라 일부 단계는 반복하거나 순서를 조정할 수 있습니다.